In [ ]:
import laspy as las
import numpy as np
import random
from skspatial.objects import Line

ASSETS_FOLDER = 'assets/'
OUTPUT_FOLDER = 'output/'
FILENAME = 'lasfile.las'
OUTPUT_FILENAME = 'output.las'
THR = 50
K = 80

In [ ]:
file = las.read(ASSETS_FOLDER + FILENAME)
print(np.unique(file.classification))

points = np.vstack((file.X, file.Y, file.Z, file.classification)).transpose()
print(len(points))

filtered_points = [point for point in points if point[3] != 6]
remaining_points = [point for point in points if point[3] == 6]
print(len(filtered_points))

classes = list(set(map(lambda point: point[3], filtered_points)))
print(len(classes))
print(classes)

N = int(len(filtered_points) * random.uniform(0.0005, 0.002))
random.shuffle(filtered_points)
N_points = filtered_points[0:N]
other_points = filtered_points[N:]

print(N)
print(len(N_points))
print(len(other_points))

In [ ]:
line_fit = Line.best_fit(list(map(lambda point: [point[0], point[1], point[2]], N_points)))
print(line_fit)

point = other_points[0]
print()

for point in list(other_points):
  distance = line_fit.distance_point([point[0], point[1], point[2]])

  if(distance < THR):
    N_points.append(point)
    other_points.remove(point)

print(len(N_points) / len(filtered_points))

In [ ]:
errors = []
N_points_array = []
filtered_points_array = []

#if len(N_points) / len(filtered_points) > 0.1:
print('FOUND FIT')
line_fit = Line.best_fit(list(map(lambda point: [point[0], point[1], point[2]], N_points)))
err = 0.0

for point in N_points:
  err += line_fit.distance_point([point[0], point[1], point[2]])

errors.append(err)
N_points_array.append(N_points)
filtered_points_array.append(filtered_points)

In [ ]:
if len(errors) > 0:
  min_index = np.argmin(errors)

  all_points = list(map(lambda point: [point[0], point[1], point[2], 12], N_points_array[min_index])) + filtered_points_array[min_index] + remaining_points

  output_file = las.LasData(file.header)
  output_file.points = file.points.copy()
  output_file.X = list(map(lambda point: point[0], all_points))
  output_file.Y = list(map(lambda point: point[1], all_points))
  output_file.Z = list(map(lambda point: point[2], all_points))
  output_file.classification = list(map(lambda point: point[3], all_points))

  output_file.write(OUTPUT_FOLDER + OUTPUT_FILENAME)